# Limpieza de Datos
En este notebook, se realiza la limpieza del dataset [Beijing Multi-Site Air-Quality Data Set][kaggle] de Kaggle


[kaggle]: https://www.kaggle.com/datasets/sid321axn/beijing-multisite-airquality-data-set

### **Objetivos**

1. Identificación de valores nulos
2. Eliminación y/o reemplazo de valores nulos (Interpolación)
3. Identificación y eliminación de valores duplicados
4. Conversión de Datos Temporales
5. Ingeniería de Datos
6. Tratamiento de Outliers
7. Estandarización de Variables
8. Generación del nuevo dataset tratado

### Importación de Librerías

In [12]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler

### Cargando el Dataset

In [2]:
df = pd.read_csv("../data/PRSA_Data_Aotizhongxin.csv")

### Búsqueda de valores NaN
A continuación se presenta la cantidad de valores nulos por cada columna.

In [3]:
df.isna().sum()

No            0
year          0
month         0
day           0
hour          0
PM2.5       925
PM10        718
SO2         935
NO2        1023
CO         1776
O3         1719
TEMP         20
PRES         20
DEWP         20
RAIN         20
wd           81
WSPM         14
station       0
dtype: int64

Doce variables del dataset presentan valores nulos:
- La variable `PM2.5` presenta 925.
- La variable `PM10` presenta 718.
- La variable `SO2` presenta 935.
- La variable `NO2` presenta 1023.
- La variable `CO` presenta 1776.
- La variable `O3` presenta 1719.
- La variable `TEMP` presenta 20.
- La variable `PRES` presenta 20.
- La variable `DEWP` presenta 209.
- La variable `RAIN` presenta 20.
- La variable `wd` presenta 81.
- La variable `WSPM` presenta 14.



### Tratamiento de Valores Nulos

#### Interpolación lineal para llenar los nulos

La interpolación es una técnica común en series temporales y es especialmente útil cuando los valores nulos son relativamente pocos en relación con el tamaño del dataset y los datos presentan una secuencia temporal continua. Dado que la cantidad de nulos en algunas columnas es pequeña comparada con el total, la interpolación lineal puede ser una buena opción para mantener la continuidad temporal sin perder información.

In [4]:
# Separar columnas numéricas y categóricas
num_cols = df.select_dtypes(include=['number']).columns
cat_cols = df.select_dtypes(exclude=['number']).columns

# Interpolación solo en las columnas numéricas
df[num_cols] = df[num_cols].interpolate(method='linear')

# Verificar si quedan valores nulos
print(df.isnull().sum())

No          0
year        0
month       0
day         0
hour        0
PM2.5       0
PM10        0
SO2         0
NO2         0
CO          0
O3          0
TEMP        0
PRES        0
DEWP        0
RAIN        0
wd         81
WSPM        0
station     0
dtype: int64


#### Búsqueda y Eliminación de Registros Duplicados

Se procede a identificar y eliminar las filas duplicadas para asegurar que cada registro en el dataset sea único y evitar algún tipo de sesgo.

In [5]:
print(f'Cantidad de filas duplicadas: {df.duplicated().sum()}')

df.drop_duplicates(inplace=True)
print(f'Número de filas duplicadas después de la limpieza: {df.duplicated().sum()}')

Cantidad de filas duplicadas: 0
Número de filas duplicadas después de la limpieza: 0


#### Conversión de Datos Temporales a Datetime
Se combina las columnas year, month, day, y hour en una sola columna datetime y se configura como índice. Esto es fundamental para análisis de series temporales.

In [6]:
# Crear la columna datetime
df['datetime'] = pd.to_datetime(df[['year', 'month', 'day', 'hour']])

# Configurar datetime como índice
df.set_index('datetime', inplace=True)
df

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
datetime,,,,,,,,,,,,,,,,,,
2013-03-01 00:00:00,1,2013,3,1,0,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Aotizhongxin
2013-03-01 01:00:00,2,2013,3,1,1,8.0,8.0,4.0,7.0,300.0,77.0,-1.1,1023.2,-18.2,0.0,N,4.7,Aotizhongxin
2013-03-01 02:00:00,3,2013,3,1,2,7.0,7.0,5.0,10.0,300.0,73.0,-1.1,1023.5,-18.2,0.0,NNW,5.6,Aotizhongxin
2013-03-01 03:00:00,4,2013,3,1,3,6.0,6.0,11.0,11.0,300.0,72.0,-1.4,1024.5,-19.4,0.0,NW,3.1,Aotizhongxin
2013-03-01 04:00:00,5,2013,3,1,4,3.0,3.0,12.0,12.0,300.0,72.0,-2.0,1025.2,-19.5,0.0,N,2.0,Aotizhongxin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-02-28 19:00:00,35060,2017,2,28,19,12.0,29.0,5.0,35.0,400.0,95.0,12.5,1013.5,-16.2,0.0,NW,2.4,Aotizhongxin
2017-02-28 20:00:00,35061,2017,2,28,20,13.0,37.0,7.0,45.0,500.0,81.0,11.6,1013.6,-15.1,0.0,WNW,0.9,Aotizhongxin
2017-02-28 21:00:00,35062,2017,2,28,21,16.0,37.0,10.0,66.0,700.0,58.0,10.8,1014.2,-13.3,0.0,NW,1.1,Aotizhongxin


### Ingeniería de Datos

En esta etapa, se realizaron transformaciones y creaciones de nuevas columnas para enriquecer el análisis temporal y estacional en el dataset de calidad del aire en Beijing. Estas nuevas variables permitirán identificar patrones temporales y estacionales que podrían influir en la variación de los contaminantes atmosféricos. A continuación, se detallan las transformaciones realizadas:

- **Día de la Semana**: Se añadió la columna `dayofweek` en formato numérico (0 = lunes, 6 = domingo) y `dayofweek_str` con el nombre del día (Monday, Tuesday, etc.). Esto facilita el análisis de patrones específicos de días de la semana.
  
- **Trimestre**: Se creó la columna `quarter`, que indica en cuál de los cuatro trimestres del año ocurre cada registro. Esta variable ayuda a detectar variaciones a nivel trimestral.

- **Estación del Año**: Se agregó la columna `season`, que clasifica cada registro en una de las cuatro estaciones del año en inglés: **Winter**, **Spring**, **Summer**, y **Autumn**. Esta clasificación es útil para analizar patrones estacionales en los datos de contaminación.

Estas nuevas columnas proporcionan un contexto temporal adicional, permitiendo un análisis más profundo sobre cómo los factores estacionales y los días de la semana pueden afectar la calidad del aire en Beijing.


In [8]:
# Día de la semana en formato numérico (0 = lunes, 6 = domingo)
df['dayofweek'] = df.index.dayofweek

# Día de la semana en formato de texto
df['dayofweek_str'] = df.index.day_name()

# Trimestre
df['quarter'] = df.index.quarter

# Función para asignar la estación según el mes
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Autumn'

# Crear la columna de estación en inglés
df['season'] = df['month'].apply(get_season)
df['season'] = df['season'].astype('category')
df

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,...,PRES,DEWP,RAIN,wd,WSPM,station,dayofweek,dayofweek_str,quarter,season
datetime,,,,,,,,,,,,,,,,,,,,,
2013-03-01 00:00:00,1,2013,3,1,0,4.0,4.0,4.0,7.0,300.0,...,1023.0,-18.8,0.0,NNW,4.4,Aotizhongxin,4,Friday,1,Spring
2013-03-01 01:00:00,2,2013,3,1,1,8.0,8.0,4.0,7.0,300.0,...,1023.2,-18.2,0.0,N,4.7,Aotizhongxin,4,Friday,1,Spring
2013-03-01 02:00:00,3,2013,3,1,2,7.0,7.0,5.0,10.0,300.0,...,1023.5,-18.2,0.0,NNW,5.6,Aotizhongxin,4,Friday,1,Spring
2013-03-01 03:00:00,4,2013,3,1,3,6.0,6.0,11.0,11.0,300.0,...,1024.5,-19.4,0.0,NW,3.1,Aotizhongxin,4,Friday,1,Spring
2013-03-01 04:00:00,5,2013,3,1,4,3.0,3.0,12.0,12.0,300.0,...,1025.2,-19.5,0.0,N,2.0,Aotizhongxin,4,Friday,1,Spring
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-02-28 19:00:00,35060,2017,2,28,19,12.0,29.0,5.0,35.0,400.0,...,1013.5,-16.2,0.0,NW,2.4,Aotizhongxin,1,Tuesday,1,Winter
2017-02-28 20:00:00,35061,2017,2,28,20,13.0,37.0,7.0,45.0,500.0,...,1013.6,-15.1,0.0,WNW,0.9,Aotizhongxin,1,Tuesday,1,Winter
2017-02-28 21:00:00,35062,2017,2,28,21,16.0,37.0,10.0,66.0,700.0,...,1014.2,-13.3,0.0,NW,1.1,Aotizhongxin,1,Tuesday,1,Winter


### Método del Rango Intercuartílico (IQR)

El método IQR es una técnica común para detectar outliers. Calcularemos el IQR para cada columna de contaminantes y se detecta los valores que están fuera del rango normal (por debajo de Q1 - 1.5 * IQR o por encima de Q3 + 1.5 * IQR).

In [10]:
# Columnas de interés para la detección de outliers
contaminants = ['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3']

# Detectar y contar outliers en cada columna
for col in contaminants:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers = df[(df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR))]
    print(f"Outliers en {col}: {len(outliers)}")

Outliers en PM2.5: 1653
Outliers en PM10: 1130
Outliers en SO2: 2864
Outliers en NO2: 560
Outliers en CO: 2695
Outliers en O3: 1399


### Tratamiento de Outliers

Para el tratamiento de los outliers, podemos tomar dos caminos: 
1. **Mantener los outliers:** Esta opción es útil en series temporales, ya que los valores extremos pueden reflejar episodios de alta contaminación que son importantes para el análisis.
2. **Recortar los outliers:** Si nuestro objetivo es suavizar la serie, podemos limitar los valores atípicos al percentil 1 y el percentil 99.

En este caso nos quedamos con la primera opción, que será mantener los outliers.

### Estandarización de las Variables
La estandarización es útil para que todas las variables tengan una media de 0 y una desviación estándar de 1, lo que facilita el rendimiento de algunos modelos de series temporales al reducir el efecto de la escala de las variables.

In [13]:
# Seleccionar columnas numéricas para estandarizar
numeric_cols = ['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'WSPM']

# Aplicar estandarización
scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

In [14]:
# Verificar media y desviación estándar después de la estandarización
print(df[numeric_cols].mean())
print(df[numeric_cols].std())

PM2.5   -3.242267e-17
PM10     7.619327e-17
SO2      1.945360e-17
NO2      3.242267e-17
CO      -2.593813e-17
O3       3.566493e-17
TEMP    -7.781440e-17
PRES     6.705007e-15
DEWP     0.000000e+00
RAIN     2.431700e-18
WSPM     6.160307e-17
dtype: float64
PM2.5    1.000014
PM10     1.000014
SO2      1.000014
NO2      1.000014
CO       1.000014
O3       1.000014
TEMP     1.000014
PRES     1.000014
DEWP     1.000014
RAIN     1.000014
WSPM     1.000014
dtype: float64


#### Dataset limpio

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 35064 entries, 2013-03-01 00:00:00 to 2017-02-28 23:00:00
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   No             35064 non-null  int64   
 1   year           35064 non-null  int64   
 2   month          35064 non-null  int64   
 3   day            35064 non-null  int64   
 4   hour           35064 non-null  int64   
 5   PM2.5          35064 non-null  float64 
 6   PM10           35064 non-null  float64 
 7   SO2            35064 non-null  float64 
 8   NO2            35064 non-null  float64 
 9   CO             35064 non-null  float64 
 10  O3             35064 non-null  float64 
 11  TEMP           35064 non-null  float64 
 12  PRES           35064 non-null  float64 
 13  DEWP           35064 non-null  float64 
 14  RAIN           35064 non-null  float64 
 15  wd             34983 non-null  object  
 16  WSPM           35064 non-null  float64 
 

#### Generación del Dataset Limpio

In [16]:
df.to_csv('../data/dataset_clean.csv', index=False)

### Conclusiones de la Limpieza, Transformación e Ingeniería de Datos

En esta etapa, se completaron diversas acciones para preparar el dataset y asegurar su adecuación para el análisis de estacionariedad. A continuación, se resumen los principales procesos realizados:

1. **Tratamiento de Valores Nulos**: Se aplicó interpolación lineal a todas las columnas numéricas para llenar valores nulos, lo cual mantiene la continuidad temporal del dataset y minimiza la pérdida de datos.
  
2. **Transformación Temporal**: Las columnas de año, mes, día y hora se combinaron en una sola columna de tipo `datetime` y se estableció como índice temporal del DataFrame. Esto facilita el análisis temporal y permite aplicar modelos de series temporales de manera eficiente.

3. **Ingeniería de Datos**: Se crearon nuevas columnas, como `dayofweek`, `quarter` y `season`, que permiten capturar patrones estacionales y variaciones según el día de la semana o el trimestre. Estas nuevas variables enriquecerán el análisis al aportar contexto adicional.

4. **Detección y Tratamiento de Outliers**: Los valores atípicos en concentraciones de contaminantes fueron detectados mediante el método del rango intercuartílico (IQR) y se dcidió no suavisar, ya que estos valores pueden ser útiles para estudios posteriores.

5. **Estandarización de Variables**: Se realizó la estandarización de variables numéricas claves, asegurando que todas las columnas tengan una media de 0 y una desviación estándar de 1. Esto es especialmente importante para modelos que dependen de la escala de los datos.

Esta etapa asegura que el dataset esté limpio, estructurado y optimizado para los siguientes análisis de series temporales, permitiendo detectar patrones y realizar predicciones de forma precisa.

